### 練習問題1.44
関数の平滑化(smoothing)は、  
信号処理において重要な概念である。  
$f$が関数で$dx$がある⼩さな値であるとき、  
$f$を平滑化したものは$x$での値が$f(x−dx), f(x), f(x + dx)$の平均となる関数である。  
⼊⼒として$f$を計算する⼿続きを取り、$f$を平滑化したものを計算する⼿続きを返す⼿続きsmoothを書け。  
時には、関数を繰り返し平滑化する(つまり、平滑化した関数をさらに平滑化することを続ける)ことによって**$n$重平滑化関数**(n-fold smoothedfunction)を得ることが役に⽴つこともある。  
smoothと練習問題1.43のrepeatedを使って任意の関数の$n$重平滑化関数を ⽣成するやり⽅を⽰せ。

回答は割愛する。